# 开机自动发送指令

本章教程用于介绍上位机每次开机后会自动执行指令和向下位机发送一些指令，本章教程的代码块不需要执行（也不能执行），仅用于理解开机后产品的一些自动操作，如果你有需要的话，更改或增加这些指令。

## cmd_on_boot() 函数

cmd_on_boot() 函数位于产品主程序 app.py 中，这个函数每次开机时会被调用，你可以编辑这个函数来对开机时自动运行的指令进行调参/增加指令。

In [ ]:
def cmd_on_boot():
    # 定义启动时要执行的命令列表
    cmd_list = [
        'base -c {"T":142,"cmd":50}',   # set feedback interval
        'base -c {"T":131,"cmd":1}',    # serial feedback flow on
        'base -c {"T":143,"cmd":0}',    # serial echo off
        'base -c {"T":300,"mode":0,"mac":"EF:EF:EF:EF:EF:EF"}',  # the base won't be ctrl by esp-now broadcast cmd, but it can still recv broadcast megs.
        'send -a -b'    # add broadcast mac addr to peer
    ]
    # 遍历命令列表
    for i in range(0, len(cmd_list)):
        cmdline_ctrl(cmd_list[i])
        cvf.info_update(cmd_list[i], (0,255,255), 0.36)
    # 设置底盘类型和模块类型
    set_version(f['base_config']['main_type'], f['base_config']['module_type'])

产品上位机可以通过命令行指令来进行一些功能方面的控制，类似上面的 base -c 指令，用于直接将后面写入的 JSON 指令通过树莓派的GPIO串口传递给下位机，后面我们会详细解释这里默认的开机自动运行的指令是什么意思。

- 'base -c {"T":142,"cmd":50}'
> 用于设置下位机连续反馈信息的格外间隔时间，cmd的值的单位为ms，此功能用于降低下位机反馈信息的频率，目的是减轻上位机处理下位机反馈信息的算力压力。
- 'base -c {"T":131,"cmd":1}'
> 开启下位机连续信息反馈功能，该功能开启后，不需要上位机一问一答地去获取下位机的信息，下位机正常会默认开启该功能，但我们这里还是再发送一次开启该功能的指令，比较保险。
- 'base -c {"T":143,"cmd":0}'
> 关闭串口指令回声，这样上位机在向下位机发送指令时，下位机不会再将接收到的指令反馈给上位机，这样可以避免上位机处理无用的信息。
- 'base -c {"T":4,"cmd":2}'
> 设置外置模块的类型，cmd的值为0时，代表没有外接模块；1，机械臂；2，云台，如果你的产品没有安装云台或机械臂，你需要把这里的数值改为0。
- 'base -c {"T":300,"mode":0,"mac":"EF:EF:EF:EF:EF:EF"}'
> 避免底盘通过其它设备的ESP-NOW广播控制，但是除mac地址的设备外，你可以自己随意编一个MAC地址，也可以使用你自己的ESP32遥控器的MAC地址。
- 'send -a -b'
> 将广播地址（FF:FF:FF:FF:FF:FF）添加到peer，这样你可以后续直接通过广播信号来将广播信息发送到其它设备。

其它的上位机命令行指令你可以通过后面的 WEB 命令行应用章节来了解。